In [1]:
import numpy as np
import pandas as pd
import math
import statsmodels.tsa.stattools as statools

In [2]:
ngonye_data='./output_data/2020/'
output_data='./output_data/forecast/'

In [3]:
ngo = pd.read_csv(ngonye_data + 'ngonye_daily.csv')
ngo['Date']=pd.to_datetime(ngo['Date'],format="%Y-%m-%d")#"%d/%m/%Y")
ngo=ngo.set_index(pd.DatetimeIndex(ngo['Date']))
ngo['DateVal']=ngo.index

In [4]:
ngo = ngo[['Flow','Exceedance','DateVal','Year','Month','Day']]
ngo

,Flow,Exceedance,DateVal,Year,Month,Day
Date,,,,,,
1924-10-01,140.730461,0.999,1924-10-01,1924,10,1
1924-10-02,140.730461,0.999,1924-10-02,1924,10,2
1924-10-03,140.730461,0.999,1924-10-03,1924,10,3
1924-10-04,140.730461,0.999,1924-10-04,1924,10,4
1924-10-05,140.730461,0.999,1924-10-05,1924,10,5
...,...,...,...,...,...,...
2020-05-16,3580.983662,0.056,2020-05-16,2020,5,16
2020-05-17,3559.103900,0.057,2020-05-17,2020,5,17
2020-05-18,3490.510716,0.059,2020-05-18,2020,5,18


In [7]:
actual = pd.read_csv(output_data + 'ngonye_forecast_actual.csv')
actual['DateVal']=pd.to_datetime(actual['DateVal'],format="%Y-%m-%d")
actual=actual.set_index('DateVal')
actual = actual.rename(columns={c: int(c) for c in actual.columns})
actual

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
DateVal,,,,,,,,,,,,,,,,,,,,,
1924-10-01,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,...,144.610705,144.610705,147.267478,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365
1924-10-02,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,...,144.610705,147.267478,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,138.899082
1924-10-03,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,...,147.267478,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,138.899082,141.399969
1924-10-04,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,...,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,138.899082,141.399969,143.928906
1924-10-05,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,...,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,138.899082,141.399969,143.928906,138.842814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-16,3559.103900,3490.510716,3390.523841,3318.867319,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-17,3490.510716,3390.523841,3318.867319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-18,3390.523841,3318.867319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
rmse=pd.DataFrame()

In [9]:
def calcRMSE(model,prefix):
    d1=model[np.arange(1,31,1)]-actual[np.arange(1,31,1)]
    d1['DateVal']=ngo['DateVal']
    d1['Flow']=ngo['Flow']
    e1=d1[np.arange(1,31,1)].divide(d1['Flow'],axis=0)
    rmse[prefix]=((e1[np.arange(1,31,1)]**2).mean()**0.5)
    #rmse=((e1[np.arange(1,31,1)]**2).mean()**0.5).to_frame().rename(columns={0:"linear_1p"})
    #rmse

In [10]:
m1=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m1.index.name='DateVal'
m1['m']=m1['Flow']-m1['Flow'].shift(1)
m1[np.arange(1,31,1)]= m1.apply(lambda x: x['Flow']+(pd.Series(np.arange(1,31,1))*x['m']),axis=1)

In [57]:
calcRMSE(m1,'linear_2p')

In [12]:
m3=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m3.index.name='DateVal'
m3['m']=(m3['Flow']-m3['Flow'].shift(4))/4
m3[np.arange(1,31,1)]= m3.apply(lambda x: x['Flow']+(pd.Series(np.arange(1,31,1))*x['m']),axis=1)

In [58]:
calcRMSE(m3,'linear_3p')

In [14]:
m4=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m4.index.name='DateVal'
m4['m']=(m4['Flow']-m4['Flow'].shift(8))/8
m4[np.arange(1,31,1)]= m4.apply(lambda x: x['Flow']+(pd.Series(np.arange(1,31,1))*x['m']),axis=1)

In [59]:
calcRMSE(m4,'linear_7p')

In [18]:
m5=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m5.index.name='DateVal'
m5['m']=m5['Flow'].rolling(3).apply(lambda r: pd.Series(np.polyfit([1,2,3],r,1)[0]))
m5['c']=m5['Flow'].rolling(3).apply(lambda r: pd.Series(np.polyfit([1,2,3],r,1)[1]))
m5[np.arange(1,31,1)]= m5.apply(lambda x: x['c']+(pd.Series(np.arange(4,34,1))*x['m']),axis=1)

In [60]:
calcRMSE(m5,'poly_d1_3p')

In [20]:
m6=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m6.index.name='DateVal'
m6['m']=m6['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,1)[0]))
m6['c']=m6['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,1)[1]))
m6[np.arange(1,31,1)]= m6.apply(lambda x: x['c']+(pd.Series(np.arange(8,38,1))*x['m']),axis=1)

In [61]:
calcRMSE(m6,'poly_d1_7p')

In [39]:
m7=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m7.index.name='DateVal'
m7['m']=m7['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,2)[0]))
m7['n']=m7['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,2)[1]))
m7['c']=m7['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,2)[2]))
m7[np.arange(1,31,1)]= m7.apply(lambda x: x['c']+((pd.Series(np.arange(8,38,1))**2)*x['m'])+(pd.Series(np.arange(8,38,1))*x['n']),axis=1)

In [62]:
calcRMSE(m7,'poly_d2_7p')

In [41]:
m8=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m8.index.name='DateVal'
m8['m']=m8['Flow'].rolling(15).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],r,2)[0]))
m8['n']=m8['Flow'].rolling(15).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],r,2)[1]))
m8['c']=m8['Flow'].rolling(15).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],r,2)[2]))
m8[np.arange(1,31,1)]= m8.apply(lambda x: x['c']+((pd.Series(np.arange(16,46,1))**2)*x['m'])+(pd.Series(np.arange(16,46,1))*x['n']),axis=1)

In [63]:
calcRMSE(m8,'poly_d2_15p')

In [29]:
#m9=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
#m9.index.name='DateVal'
#m9['m']=m9['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,3)[0]))
#m9['n']=m9['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,3)[1]))
#m9['o']=m9['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,3)[2]))
#m9['c']=m9['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,3)[3]))
#m9[np.arange(1,31,1)]= m9.apply(lambda x: x['c']+((pd.Series(np.arange(8,38,1))**3)*x['m'])+((pd.Series(np.arange(8,38,1))**2)*x['n'])+(pd.Series(np.arange(8,38,1))*x['o']),axis=1)

In [52]:
#calcRMSE(m9,'poly_d3_7p')

In [35]:
m10=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m10.index.name='DateVal'
m10['m']=m10['Flow'].rolling(15).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],r,3)[0]))
m10['n']=m10['Flow'].rolling(15).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],r,3)[1]))
m10['o']=m10['Flow'].rolling(15).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],r,3)[2]))
m10['c']=m10['Flow'].rolling(15).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],r,3)[3]))
m10[np.arange(1,31,1)]= m10.apply(lambda x: x['c']+((pd.Series(np.arange(16,46,1))**3)*x['m'])+((pd.Series(np.arange(16,46,1))**2)*x['n'])+(pd.Series(np.arange(16,46,1))*x['o']),axis=1)

In [64]:
calcRMSE(m10,'poly_d3_15p')

In [43]:
m11=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m11.index.name='DateVal'
m11['m']=m11['Flow'].rolling(21).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],r,3)[0]))
m11['n']=m11['Flow'].rolling(21).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],r,3)[1]))
m11['o']=m11['Flow'].rolling(21).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],r,3)[2]))
m11['c']=m11['Flow'].rolling(21).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],r,3)[3]))
m11[np.arange(1,31,1)]= m11.apply(lambda x: x['c']+((pd.Series(np.arange(22,52,1))**3)*x['m'])+((pd.Series(np.arange(22,52,1))**2)*x['n'])+(pd.Series(np.arange(22,52,1))*x['o']),axis=1)

In [65]:
calcRMSE(m11,'poly_d3_21p')

In [66]:
rmse

,linear_2p,linear_3p,linear_7p,poly_d1_3p,poly_d1_7p,poly_d2_7p,poly_d2_15p,poly_d3_15p,poly_d3_21p
1,0.023976,0.020780,0.020657,0.022702,0.026739,0.026976,0.030301,0.030799,0.032405
2,0.044498,0.036001,0.035570,0.039560,0.040651,0.050219,0.046565,0.056193,0.053424
3,0.065037,0.051348,0.050610,0.056550,0.055352,0.077538,0.064469,0.087785,0.078121
4,0.086221,0.067598,0.066668,0.074489,0.071221,0.109465,0.084311,0.126816,0.107182
5,0.108376,0.084853,0.083789,0.093535,0.088113,0.146147,0.106019,0.174109,0.140921
6,0.131265,0.102894,0.101703,0.113164,0.105846,0.187301,0.129433,0.230354,0.179615
7,0.154444,0.121569,0.120233,0.133347,0.124384,0.232891,0.154552,0.296359,0.223684
8,0.178278,0.140991,0.139411,0.154340,0.143646,0.283231,0.181440,0.372910,0.273530
9,0.202742,0.161036,0.159112,0.175796,0.163395,0.338450,0.209995,0.460720,0.329396
10,0.227450,0.181570,0.179233,0.197721,0.183549,0.398579,0.240209,0.560627,0.391675


In [67]:
rmse.to_csv(output_data + 'forecast_model_errors.csv')

In [32]:
def split_and_save(model,prefix):
    model['Year']=model.index.year
    model['Month']=model.index.month
    model['Day']=model.index.day
    model['WaterYear']=model.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
    years=model['WaterYear'].unique().tolist()

    for year in years:
        days=model.loc[model.WaterYear==year]
        days.to_csv(output_data + prefix + '_' + str(year) + '.csv',float_format='%.0f')

In [68]:
split_and_save(m1,'linear_2p')
split_and_save(m3,'linear_3p')
split_and_save(m4,'linear_7p')
split_and_save(m5,'poly_d1_3p')
split_and_save(m6,'poly_d1_7p')
split_and_save(m7,'poly_d2_7p')
split_and_save(m8,'poly_d2_15p')
#split_and_save(m9,'poly_d3_7p')
split_and_save(m10,'poly_d3_15p')
split_and_save(m11,'poly_d3_21p')

## Actual calc

In [ ]:
actual=pd.DataFrame(columns=np.arange(1,31,1))
actual['DateVal']=[]
actual

for index, row in ngo.reset_index().iterrows():

    vals=pd.Series(ngo['Flow'].iloc[index+1:index+31]).reset_index().drop('Date',axis=1)
    vals=vals.rename(columns={"Flow":row['DateVal']})
    vals.index+=1
    vals=vals.transpose().reset_index().rename(columns={"index":"DateVal"})
    if index % 1000 == 0:
        display(index)
    actual=pd.concat([actual,vals],axis=0)

actual=actual.reset_index().set_index('DateVal')
actual=actual.drop('index',axis=1) 
actual.to_csv(ngonye_data + 'ngonye_forecast_actual.csv')